# Network analysis in Senegal

### Objectives
    1)	Use measures of road-based accessibility to identify road segments that, if rehabilitated, would improve agricultural market activities in Senegal, including during flood conditions.
    2)	Gain a better understanding of the accessibility, connectivity, and criticality of roads in Senegal in relationship to agricultural origins, processing & transfer sites, and markets.

To this end, the team will develop an accessibility model which measures the travel time from sites of agricultural production to their nearest populated areas, processing centers, and markets. 

### Datasets for analysis
#### ORIGIN
    1) agriculture: MapSPAM 2017. Measuring value in international dollars.
    2) agriculture: UMD Land Cover 2019 30m. Assign MapSPAM value onto land cover cropland class for more precise origin information.
    3) population: WorldPop 2020, UN-adjusted.
    4) settlement extent: GRID3 2020.
#### DESTINATION
    4) markets: derived from WorldPop 2020 and GRID3 2020 urban clusters.
    5) agricultural processing hubs: to be acquired.
#### TRAVEL ROUTE
    6) roads: OpenStreetMap, July 2021.
    7) elevation: 
#### OBSTACLE
    8) flood: FATHOM. 1-in-10, 20, and 50 year flood return periods. 
#### INTERVENTION
    9) upcoming road projects: AGEROUTE interventions separate from the World Bank-financed project
    10) targeted road projects: critical road segments identified by this accessibility model's baseline outputs


### Model design
#### Basic formula: 
    (a) Off-road driving time from origin to closest road node
    +
    (b) Driving time from road node in (a) to a destination (closeness measured by road segments speeds)

#### Model origin & destination (OD) sets:
    A)	Travel time from an area that has agricultural value/potential to the nearest processing hub (if provided).
    B)	Travel time from an area that has agricultural value/potential to the nearest larger settlement, (“larger” settlement identified using a case-appropriate population metric to be determined).
    C)	Travel time from an area that has agricultural value/potential to the nearest market.
    D)	Travel time from all settlements to the nearest market.
    E)	Travel time from larger settlements to the nearest market.

#### Before/after scenarios for each OD set:
    1)	Pre-project, baseline weather: No inclement weather. Road network status as of November 2021.
    2)	Pre-project, flood: 1-in-10, 1-in-20 and 1-in-50 year flood return period. Road network status as of November 2021.
    3)	Post-project, baseline weather: No inclement weather. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).
    4)	Post-project, flood: 1-in-10 year flood return period. Road network status if X number of critical road segments to high-value areas are protected (i.e., their travel times reduced).

#### Notes:
    --Destinations are expected to be proximal to the road network, so no measure is taken between road and destination.
    --All travel times will be assigned to each model variation’s point of origin; the aggregation up to admin areas is possible if desired.
    --Obstacles & interventions modify the road segment speeds. Basic formula is then applied to the modified road network.


### Prep workspace

In [1]:
import os, sys
GISFolder = os.getcwd()
GISFolder

'C:\\Users\\wb527163\\GEO-Cdrive-Grace'

In [2]:
# Note: needed to reinstall rtree due to geopandas import error. Did so in the console. 
# conda install -c conda-forge rtree=0.9.3

In [3]:
# load and filter osm network (step 1)
import geopandas as gpd
from geopandas import GeoDataFrame
import pandas as pd
import time
sys.path.append(r"C:\Users\wb527163\.conda\envs\geo\GOSTnets-master")
import GOSTnets as gn

In [4]:
import networkx as nx
import osmnx as ox
import numpy as np
import rasterio as rt
import shapely
from shapely.geometry import Point, box
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely import wkt
from shapely.geometry import LineString, MultiLineString, Point
import peartree

In [5]:
#### Might not use these
import fiona
from osgeo import gdal
import importlib
import matplotlib.pyplot as plt
import subprocess, glob

In [6]:
pth = os.path.join(GISFolder, "SEN-Cdrive") # Personal folder system for running model.
pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive'

In [7]:
out_pth = os.path.join(GISFolder, "SEN-Cdrive\outputs") # For storing intermediate outputs from the model.
out_pth

'C:\\Users\\wb527163\\GEO-Cdrive-Grace\\SEN-Cdrive\\outputs'

In [8]:
team_pth = 'R:\\SEN\\GEO' # This is where the unmodified input data is stored. Finalized outputs also housed here.
team_pth

'R:\\SEN\\GEO'

### Join back with full georeferenced origins and combine with cost-distance raster travel times from origins to road node.

#### Hamlets

In [24]:
hamlet_CD = gpd.read_file("R:\SEN\GEO\Team\Projects\Sen_TransportOV\SEN_TransportOV.gdb", layer="hamlet_pt") # Contains cost-distance travel from origin to road node.
hamlet_CD = pd.DataFrame(hamlet_CD)
hamlet_snap = os.path.join(out_pth, "hamlet_snap.csv") # Georeferenced to origin. Contains hamlet ID and road node ID.
hamlet_snap = pd.read_csv(hamlet_snap)

print(hamlet_CD.info(), end="\n")
print(hamlet_snap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   Unnamed__0  125886 non-null  float64 
 1   mgrs_code   125886 non-null  object  
 2   type        125886 non-null  object  
 3   GlobalID    125886 non-null  object  
 4   Shape_Leng  125886 non-null  float64 
 5   CD_gNodesP  125402 non-null  float64 
 6   geometry    125886 non-null  geometry
dtypes: float64(3), geometry(1), object(3)
memory usage: 6.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125886 entries, 0 to 125885
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  125886 non-null  int64  
 1   Unnamed_ 0  125886 non-null  int64  
 2   mgrs_code   125886 non-null  object 
 3   type        125886 non-null  object 
 4   GlobalID    125886 non-null  object 
 5   Shape_Leng  125886 non-

In [25]:
hamletbase = os.path.join(out_pth, "hamlet_to_HDurban_drive_pre-project.csv")
hamletbase = pd.read_csv(hamletbase)
hamletF10 = os.path.join(out_pth, "hamlet_to_HDurban_drive_pre-project_flood10.csv")
hamletF10 = pd.read_csv(hamletF10)
hamletF20 = os.path.join(out_pth, "hamlet_to_HDurban_drive_pre-project_flood20.csv")
hamletF20 = pd.read_csv(hamletF20)
hamletF50 = os.path.join(out_pth, "hamlet_to_HDurban_drive_pre-project_flood50.csv")
hamletF50 = pd.read_csv(hamletF50)

print(hamletbase.info(), end="\n")
print(hamletF10.info(), end="\n")
print(hamletF20.info(), end="\n")
print(hamletF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74662 entries, 0 to 74661
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  74662 non-null  int64  
 1   NN          74662 non-null  int64  
 2   ha_HD1      74661 non-null  float64
 3   ha_HD2      74662 non-null  float64
 4   ha_HD3      74662 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 2.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74662 entries, 0 to 74661
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  74662 non-null  int64  
 1   NN          74662 non-null  int64  
 2   ha_HD1      74661 non-null  float64
 3   ha_HD2      74662 non-null  float64
 4   ha_HD3      74662 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 2.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74662 entries, 0 to 74661
Data columns (total 5 co

In [26]:
# Make sure the ID is unique and fully matched
list1 = list(hamletbase['NN'].unique())
list2 = list(hamletF10['NN'].unique())
list3 = list(hamletF20['NN'].unique())
list4 = list(hamletF50['NN'].unique())
list5 = list(hamlet_snap['NN'].unique())
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))
print(len(list5))

list6 = list(hamlet_snap['mgrs_code'].unique())
list7 = list(hamlet_CD['mgrs_code'].unique())
print(len(list6))
print(len(list7))

74662
74662
74662
74662
74662
125886
125886


In [27]:
hamletF10.rename(columns={'ha_HD1': 'HD1F10'}, inplace=True)
hamletF20.rename(columns={'ha_HD1': 'HD1F20'}, inplace=True)
hamletF50.rename(columns={'ha_HD1': 'HD1F50'}, inplace=True)
hamletbase = hamletbase[['NN', 'ha_HD1']]
hamletF10 = hamletF10[['NN', 'HD1F10']]
hamletF20 = hamletF20[['NN', 'HD1F20']]
hamletF50 = hamletF50[['NN', 'HD1F50']]
hamlet_snap = hamlet_snap[['mgrs_code', 'geometry', 'NN']]
hamlet_CD = hamlet_CD[['mgrs_code', 'CD_gNodesP']]

In [28]:
hamlet_all = pd.merge(hamlet_snap, hamlet_CD, on='mgrs_code',how='left')
del hamlet_snap, hamlet_CD # Avoiding memory issues.

In [29]:
hamlet_all = pd.merge(hamlet_all, hamletbase, on='NN',how='left')
del hamletbase

In [30]:
hamlet_all = pd.merge(hamlet_all, hamletF10, on='NN',how='left')
del hamletF10

In [31]:
hamlet_all = pd.merge(hamlet_all, hamletF20, on='NN',how='left')
del hamletF20

In [32]:
hamlet_all = pd.merge(hamlet_all, hamletF50, on='NN',how='left')
del hamletF50
print(hamlet_all.info(), end='\n\n')
print(hamlet_all.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   mgrs_code   125886 non-null  object 
 1   geometry    125886 non-null  object 
 2   NN          125886 non-null  int64  
 3   CD_gNodesP  125402 non-null  float64
 4   ha_HD1      125885 non-null  float64
 5   HD1F10      125885 non-null  float64
 6   HD1F20      125885 non-null  float64
 7   HD1F50      125885 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 8.6+ MB
None

      mgrs_code                                        geometry          NN  \
0  28PCU1265_01  POINT (-16.721473282454415 12.348636090165247)  7761872870   
1  28PCU1365_01   POINT (-16.716456507935607 12.34788723564788)  7761872869   
2  28PCU1365_02  POINT (-16.713855008830738 12.350880992129111)  6442044321   
3  28PCU1566_01  POINT (-16.701275174874546 12.355585269999269)  2142496418   
4  28PC

In [33]:
hamlet_all['pre'] = 0
hamlet_all['pre'] = hamlet_all['ha_HD1'] + hamlet_all['CD_gNodesP']
hamlet_all['F10'] = 0
hamlet_all['F10'] = hamlet_all['HD1F10'] + hamlet_all['CD_gNodesP']
hamlet_all['F20'] = 0
hamlet_all['F20'] = hamlet_all['HD1F20'] + hamlet_all['CD_gNodesP']
hamlet_all['F50'] = 0
hamlet_all['F50'] = hamlet_all['HD1F50'] + hamlet_all['CD_gNodesP']

In [34]:
# Remove legacy variables.
hamlet_all = hamlet_all.drop(columns=['ha_HD1', 'HD1F10', 'HD1F20', 'HD1F50'])
hamlet_all.rename(columns={'CD_gNodesP': 'offroad'}, inplace=True)
hamlet_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   geometry   125886 non-null  object 
 2   NN         125886 non-null  int64  
 3   offroad    125402 non-null  float64
 4   pre        125401 non-null  float64
 5   F10        125401 non-null  float64
 6   F20        125401 non-null  float64
 7   F50        125401 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 8.6+ MB


In [35]:
display(hamlet_all.describe())

,NN,offroad,pre,F10,F20,F50
count,1.258860e+05,125402.000000,125401.000000,125401.000000,125401.000000,125401.000000
mean,5.726621e+09,30.850911,108.466988,1302.139275,2491.133466,4756.493252
std,2.427068e+09,52.353222,91.342579,8607.901375,12680.066151,19174.814081
min,6.088434e+07,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.774781e+09,4.030059,45.322884,53.724500,60.838078,64.444047
50%,5.077606e+09,11.869895,79.700834,111.930846,160.740170,208.388311
75%,8.366050e+09,36.594818,146.106710,258.282943,340.401154,3383.050269
max,9.393948e+09,1153.522705,1239.946035,295191.841852,354801.776081,476904.241332


In [36]:
hamlet_all["geometry"] = hamlet_all["geometry"].astype('str')
hamlet_all["geometry"]  = hamlet_all["geometry"] .str.strip('POINT ')
hamlet_all["geometry"]  = hamlet_all["geometry"] .str.strip('()')
XY = hamlet_all["geometry"] .str.split(" ", expand=True)
hamlet_all["X"] = XY[0]
hamlet_all["Y"] = XY[1]
hamlet_all["X"] = hamlet_all["X"].astype('float')
hamlet_all["Y"] = hamlet_all["Y"].astype('float')
hamlet_all = hamlet_all.drop(columns=['geometry'])
hamlet_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125886 entries, 0 to 125885
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   mgrs_code  125886 non-null  object 
 1   NN         125886 non-null  int64  
 2   offroad    125402 non-null  float64
 3   pre        125401 non-null  float64
 4   F10        125401 non-null  float64
 5   F20        125401 non-null  float64
 6   F50        125401 non-null  float64
 7   X          125886 non-null  float64
 8   Y          125886 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 9.6+ MB


In [37]:
hamlet_all.to_csv(os.path.join(out_pth, 'hamlet_to_HDurban_pre-F10-20-50.csv'))

In [23]:
# To prevent memory errors if moving straight on to next set, delete hamlet files from memory
del hamlet_all

#### Agriculture

In [44]:
ag_CD = gpd.read_file("R:\SEN\GEO\Team\Projects\Sen_TransportOV\SEN_TransportOV.gdb", layer="agriculture_allorigins") # Contains cost-distance travel from origin to road node.
ag_CD = pd.DataFrame(ag_CD) 
ag_snap = os.path.join(out_pth, "ag_snap.csv") # Georeferenced to origin. Contains hamlet ID and road node ID.
ag_snap = pd.read_csv(ag_snap)

print(ag_CD.info())
print(ag_snap.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 10 columns):
 #   Column               Dtype   
---  ------               -----   
 0   ID_ag                int64   
 1   ID_spam              float64 
 2   grid_val             float64 
 3   ID_LC                float64 
 4   val                  float64 
 5   x                    float64 
 6   y                    float64 
 7   NSnomax              float64 
 8   CD_gNodesPreproj_MX  float64 
 9   geometry             geometry
dtypes: float64(8), geometry(1), int64(1)
memory usage: 1.0 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13941124 entries, 0 to 13941123
Data columns (total 12 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   ID_ag       int64  
 2   ID_spam     float64
 3   grid_val    float64
 4   ID_LC       float64
 5   val         float64
 6   x           float64
 7   y           float64
 8   NSnomax     float64
 9   ge

In [45]:
# If reloading:
agbase = os.path.join(out_pth, "ag_to_HDurban_drive_post-project.csv")
agbase = pd.read_csv(agbase)
agF10 = os.path.join(out_pth, "ag_to_HDurban_drive_post-project_flood10.csv")
agF10 = pd.read_csv(agF10)
agF20 = os.path.join(out_pth, "ag_to_HDurban_drive_post-project_flood20.csv")
agF20 = pd.read_csv(agF20)
agF50 = os.path.join(out_pth, "ag_to_HDurban_drive_post-project_flood50.csv")
agF50 = pd.read_csv(agF50)

print(agbase.info(), end="\n")
print(agF10.info(), end="\n")
print(agF20.info(), end="\n")
print(agF50.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967552 entries, 0 to 967551
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  967552 non-null  int64  
 1   NN          967552 non-null  int64  
 2   ag_HD1      967550 non-null  float64
 3   ag_HD2      967552 non-null  float64
 4   ag_HD3      967552 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 36.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967552 entries, 0 to 967551
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  967552 non-null  int64  
 1   NN          967552 non-null  int64  
 2   ag_HD1      967550 non-null  float64
 3   ag_HD2      967552 non-null  float64
 4   ag_HD3      967552 non-null  float64
dtypes: float64(3), int64(2)
memory usage: 36.9 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 967552 entries, 0 to 967551
Da

In [46]:
# Make sure the ID is unique and fully matched
list1 = list(agbase['NN'].unique())
list2 = list(agF10['NN'].unique())
list3 = list(agF20['NN'].unique())
list4 = list(agF50['NN'].unique())
list5 = list(ag_snap['NN'].unique())
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))
print(len(list5))

list6 = list(ag_snap['ID_ag'].unique())
list7 = list(ag_CD['ID_ag'].unique())
print(len(list6))
print(len(list7))

967552
967552
967552
967552
967552
13941124
13941124


In [47]:
agF10.rename(columns={'ag_HD1': 'HD1F10'}, inplace=True)
agF20.rename(columns={'ag_HD1': 'HD1F20'}, inplace=True)
agF50.rename(columns={'ag_HD1': 'HD1F50'}, inplace=True)
agbase = agbase[['NN', 'ag_HD1']]
agF10 = agF10[['NN', 'HD1F10']]
agF20 = agF20[['NN', 'HD1F20']]
agF50 = agF50[['NN', 'HD1F50']]
ag_snap = ag_snap[['ID_ag', 'geometry', 'NN']]
ag_CD = ag_CD[['ID_ag', 'val', 'CD_gNodesPreproj_MX']]

In [48]:
ag_all = pd.merge(ag_snap, ag_CD, on='ID_ag',how='left')
del ag_snap, ag_CD # Avoid memory issues.

In [49]:
ag_all = pd.merge(ag_all, agbase, on='NN',how='left')
del agbase

In [50]:
ag_all = pd.merge(ag_all, agF10, on='NN',how='left')
del agF10

In [51]:
ag_all = pd.merge(ag_all, agF20, on='NN',how='left')
del agF20

In [52]:
ag_all = pd.merge(ag_all, agF50, on='NN',how='left')
del agF50

In [53]:
ag_all['post'] = 0
ag_all['post'] = ag_all['ag_HD1'] + ag_all['CD_gNodesPreproj_MX']
ag_all['F10'] = 0
ag_all['F10'] = ag_all['HD1F10'] + ag_all['CD_gNodesPreproj_MX']
ag_all['F20'] = 0
ag_all['F20'] = ag_all['HD1F20'] + ag_all['CD_gNodesPreproj_MX']
ag_all['F50'] = 0
ag_all['F50'] = ag_all['HD1F50'] + ag_all['CD_gNodesPreproj_MX']
ag_all.head()

,ID_ag,geometry,NN,val,CD_gNodesPreproj_MX,ag_HD1,HD1F10,HD1F20,HD1F50,post,F10,F20,F50
0,13941103,POINT (-16.458987999999977 12.208646000000044),3507831609,2008851.0,NaN,82.054823,82.054823,82.054823,82.363044,NaN,NaN,NaN,NaN
1,13941122,POINT (-16.375654999999938 12.208646000000044),3507831510,94605.0,NaN,88.413319,88.413319,88.413319,88.721541,NaN,NaN,NaN,NaN
2,13941017,POINT (-16.208988999999974 12.208646000000044),6188134127,708413.0,NaN,23.536822,23.536822,23.536822,23.536822,NaN,NaN,NaN,NaN
3,13941036,POINT (-16.20849199999998 12.152815000000032),3507831510,315719.0,NaN,88.413319,88.413319,88.413319,88.721541,NaN,NaN,NaN,NaN
4,13941044,POINT (-16.294212999999957 12.149487000000022),3507831510,375288.0,NaN,88.413319,88.413319,88.413319,88.721541,NaN,NaN,NaN,NaN


In [54]:
# Remove legacy variables.
ag_all = ag_all.drop(columns=['ag_HD1', 'HD1F10', 'HD1F20', 'HD1F50'])
ag_all.rename(columns={'CD_gNodesPreproj_MX': 'offroad'}, inplace=True)
ag_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   ID_ag     int64  
 1   geometry  object 
 2   NN        int64  
 3   val       float64
 4   offroad   float64
 5   post      float64
 6   F10       float64
 7   F20       float64
 8   F50       float64
dtypes: float64(6), int64(2), object(1)
memory usage: 1.0+ GB


In [55]:
display(ag_all.describe())

,ID_ag,NN,val,offroad,post,F10,F20,F50
count,1.394112e+07,1.394112e+07,1.394112e+07,1.316869e+07,1.316869e+07,1.316869e+07,1.316869e+07,1.316869e+07
mean,6.970562e+06,5.591440e+09,1.219702e+02,2.335968e+01,7.603394e+01,1.094372e+03,1.847466e+03,3.805840e+03
std,4.024456e+06,2.392587e+09,3.760062e+03,3.347875e+01,4.788878e+01,8.386105e+03,1.180401e+04,2.236732e+04
min,0.000000e+00,2.213272e+07,1.563126e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.485281e+06,3.743274e+09,4.293236e+01,3.998711e+00,4.359728e+01,5.321674e+01,5.971764e+01,6.417238e+01
50%,6.970562e+06,4.623208e+09,7.008440e+01,1.093603e+01,6.783970e+01,8.915524e+01,1.086822e+02,1.514637e+02
75%,1.045584e+07,8.071363e+09,1.013998e+02,2.950466e+01,9.742060e+01,1.403246e+02,1.973325e+02,5.097665e+02
max,1.394112e+07,9.395831e+09,6.939670e+06,1.705327e+03,1.793787e+03,3.687666e+05,4.346180e+05,5.309666e+05


In [56]:
ag_all["geometry"] = ag_all["geometry"].astype('str')
ag_all["geometry"]  = ag_all["geometry"] .str.strip('POINT ')
ag_all["geometry"]  = ag_all["geometry"] .str.strip('()')
XY = ag_all["geometry"] .str.split(" ", expand=True)
ag_all["X"] = XY[0]
ag_all["Y"] = XY[1]
ag_all["X"] = ag_all["X"].astype('float')
ag_all["Y"] = ag_all["Y"].astype('float')
ag_all = ag_all.drop(columns=['geometry'])
ag_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13941124 entries, 0 to 13941123
Data columns (total 10 columns):
 #   Column   Dtype  
---  ------   -----  
 0   ID_ag    int64  
 1   NN       int64  
 2   val      float64
 3   offroad  float64
 4   post     float64
 5   F10      float64
 6   F20      float64
 7   F50      float64
 8   X        float64
 9   Y        float64
dtypes: float64(8), int64(2)
memory usage: 1.1 GB


In [57]:
ag_all.to_csv(os.path.join(out_pth, 'ag_to_HDurban_post-F10-20-50.csv'))

In [58]:
# To prevent memory errors if moving straight on to next set, delete hamlet files from memory
del ag_all